In [65]:
import numpy as np

import text_processing

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.naive_bayes import ComplementNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn import metrics

In [2]:
TRAINING_DATA = 'training_data/NAACL_revised.csv'

X, y = text_processing.load_and_prepare_data(TRAINING_DATA)

Problem with hashtag #MrBurns???... skipping.


In [72]:
WEIGHTS_FILE = 'embedding_weights.npy'

embedding_matrix = np.load(WEIGHTS_FILE)

def transform(x):
    emb =  embedding_matrix[x]
    return emb.reshape(len(emb), -1)

transformer = FunctionTransformer(transform, validate=False)

In [73]:
clf = Pipeline([
    ('embedding', transformer),
    ('sclaler', MinMaxScaler()),
    ('naive_bayes', ComplementNB(alpha=2))
])

In [74]:
np.random.seed(152)
X_train, X_test, y_train, y_test = train_test_split(X, y)

clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('embedding', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function transform at 0x000001687BB84C80>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('sclaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('naive_bayes', ComplementNB(alpha=2, class_prior=None, fit_prior=True, norm=False))])

In [75]:
print(metrics.classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.58      0.69      2055
           1       0.38      0.71      0.49       743

   micro avg       0.61      0.61      0.61      2798
   macro avg       0.61      0.64      0.59      2798
weighted avg       0.72      0.61      0.64      2798



In [76]:
gbdt = Pipeline([
    ('embedding', transformer),
    ('gbdt', GradientBoostingClassifier())
])

%time gbdt.fit(X_train, y_train)

Wall time: 56 s


Pipeline(memory=None,
     steps=[('embedding', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function transform at 0x000001687BB84C80>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('gbdt', GradientBoostingClassifier(criterion='fr...    subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))])

In [79]:
print(metrics.classification_report(y_test, gbdt.predict(X_test)))

              precision    recall  f1-score   support

           0       0.84      0.97      0.90      2055
           1       0.87      0.50      0.64       743

   micro avg       0.85      0.85      0.85      2798
   macro avg       0.86      0.74      0.77      2798
weighted avg       0.85      0.85      0.83      2798



In [70]:
forest = Pipeline([
    ('embedding', transformer),
    ('gbdt', RandomForestClassifier(n_estimators=50, n_jobs=-1))
])

%time forest.fit(X_train, y_train)

Wall time: 3.61 s


Pipeline(memory=None,
     steps=[('embedding', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function transform at 0x000001687BE5CD90>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=False)), ('gbdt', RandomForestClassifier(bootstrap=True, c..._jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [71]:
print(metrics.classification_report(y_test, forest.predict(X_test)))

              precision    recall  f1-score   support

           0       0.81      0.98      0.88      2055
           1       0.85      0.35      0.50       743

   micro avg       0.81      0.81      0.81      2798
   macro avg       0.83      0.67      0.69      2798
weighted avg       0.82      0.81      0.78      2798

